In [248]:
import pandas as pd
import numpy as np
jobs = ['Data scientist', 'Python', 'JS', 'Java','Data scientist', 'Python', 'JS', 'Java']
countrys = ['France','France','France','France', 'USA','USA','USA','USA']
filenames = ['data/'+str.lower(job).replace(' ','')+str.lower(country)+'.csv' for job, country in zip(jobs,countrys) ]
df = pd.concat([pd.read_csv(filename, sep=';', encoding="utf-8-sig") for filename in filenames])
df2 = df.drop_duplicates(subset=None, keep="first")

In [249]:
skills = ['Python', 'R', 'SQL', 'NoSQL', 'GIT', 'Spark', 'Flask', 'Streamlit', 'Docker', 'Kubernetes', 'React', 'Machine-Learning', 'Deep Learning', 'NLP', 'Vue', 'Angular', 'Scala', 'PySpark','PowerBI', 'SQLServer', 'Dataiku','Data', 'Keras', 'Tensor-Flow', 'NLU', 'PyTorch', 'Scikit-Learn', 'SAAS', 'Java', 'Scikit learn', 'Hadoop', 'Hive', 'ML','DL', 'Azure', 'AWS']

# contrats = ['CDI','CDD','Apprentissage','Contrat pro','Stage','Intérim','Freelance','Indépendant','Freelance']
            
studylevel_keywords = {'High School':['High School','Baccalauréat'],
                       'Guaduate': ['Université','Universitaire','Formation supérieure','Guaduate','Bac +2','Bac +3'],
                       'Bachelor': ['Bachelor'],
                       'Master': ['Master','Master 2', 'Master +2', 'Bac +5', 'Grande école d\'ingénieur','Bac +4/5','M2','Cursus ingénieur', 'engineer','Diplôme ingénieur'],
                       'Doctor': ['Doctorat', 'Doctor'], 
                       'Formation': ['Formation Data Science']
                       }

explevel_keywords = {'Beginer':['Débutant', 'Beginer','No experience','Pas d`\'expérience', '1 an', '0-1 an','0 - 1 an', '0/1 an','0 / 1 an'],
                       'Intermediate': ['Intermédiaire','Intermediate','2 year', '3 year','2-3 year',  '2 ans', '2/3 an', '1-2 an','1-3  years'],
                       'Confirmed': ['Confirmé','Confirmed','3 an','4 an','5 an','3 year','4 year','5 year','3-5 year','3-5 an','3/5 year', '3/5 an'],
                       'Expert': ['Expert','6 year','7 year', '8 year', '9 year','6 an','7 an', '8 an','9 an','10 an','5-10 year','5-10 an','5 - 10 an','5/10 year', '5/10 an'],
                       'Senoir': ['Senoir', '10+ years']
                       }

In [250]:
def exist_in_array(x, dicto):

    for key in dicto:
        if any(str.lower(term) in str.lower(x) for term in dicto[key]):
            return key

In [251]:
df2['Study required'] = df2['Description'].apply(lambda x: exist_in_array(x, studylevel_keywords))
df2['Experience required'] = df2['Description'].apply(lambda x: exist_in_array(x, explevel_keywords))

In [252]:
df2['Experience required'].unique()

array([None, '1-2  years', '3-5 years', '5-10 years', 'No experience'],
      dtype=object)

In [253]:
df2.drop('Type de contrat',axis=1, inplace=True)

In [254]:
df2

,Poste,Entreprise,Pays,Ville,Salaire,Description,Study required,Experience required
0,Alternance - Data Scientist H/F,Malakoff Humanis,France,Paris 9e (75),None,Contrat ALTERNANCE Finalité du poste Au sein d...,None,None
1,Data Scientist H/F,AGYLIS,France,La Défense (92),40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...,Master,1-2 years
2,DATA SCIENTIST- F/H,Ingeniance,France,La Défense (92),None,"Contexte: Vous assurez la production, la conce...",Master,None
3,Data Scientist and Innovation Engineer H/F,HIPAY,France,Levallois-Perret (92),None,Notre métier HiPay est une Fintech Européenne ...,Master,None
4,ASSISTANT DATA SCIENTIST,Total,France,Nanterre (92),None,Monter un système de reporting pour la directi...,Guaduate,3-5 years
...,...,...,...,...,...,...,...,...
735,Senior Developer,VirtualMGA,USA,United States,None,Virtual MGA is searching for someone who can w...,Master,None
737,"Senior Backend Engineer, Remote Assistance",Cruise,USA,"San Francisco, CA 94103 (South of Market area)",None,"We're Cruise, the self-driving ride-hailing se...",Bachelor,None
740,Principal Software Engineer/Architect,Microsoft,USA,"Redmond, WA",None,Are you interested in joining a startup within...,Master,None
742,Sr. Software Engineer,"Homecare Homebase, LLC",USA,"Louisville, KY",None,Position Summary A Senior Software Engineer wo...,Bachelor,5-10 years


In [257]:
df2.iloc[3]['Description']

"Notre métier HiPay est une Fintech Européenne spécialisée dans les solutions de paiement Omnicanal. Acteur innovant et en recherche constante d’amélioration, nous accompagnons plus de 4 000 marchands en France et à l’international. L’idée est simple: proposer une offre globale, performante et innovante permettant à nos marchands d’offrir un parcours d’achat fluide et une expérience de paiement agréable. Bien plus qu’une prestation de paiement: nous exploitons la puissance de la donnée liée à la transaction et permettons à nos marchands d’avoir une connaissance approfondie de ses clients. Click & Collect, web-to-store/store-to-web : vous nous avez peut-être utilisé récemment sans le savoir! Pour en savoir plus: https://hipay.com/ Notre identité Résolument digitale et innovante. HiPay se définit à travers un collectif débordant d’énergie, où les mots “agilité”, “bienveillance”, “autonomie” ou encore “partage” prennent tout leur sens. Ce que nous proposons Rejoindre une entreprise en ple

In [176]:
df2['Study required'].isna().sum()
# def study_level_required(x):

#     x_lower =  str.lower(x)
#     for level in studylevels:
#         leve_low = str.lower(level)
#         if leve_low in x_lower or leve_low.replace(' ','') in x_lower:
#             return level
#     return np.NaN

# df2['Study level required'] = df2['Description'].apply(lambda x : study_level_required(x))

2168

In [177]:
def skill_required(row):

    desc_low = str.lower(row['Description'])
    
    for skill in skills:

        skill_low = str.lower(skill)

        if (skill_low in desc_low) or (skill_low.replace(' ','') in desc_low) or (skill_low.replace('-','') in desc_low)  or (skill_low.replace('-',' ') in desc_low):
            row[skill] = 1

    return row

In [178]:
df2 = df2.apply(lambda row: skill_required(row), axis=1)

In [163]:
# langages_en = ['Spanish','Chinese', 'Arab','French','Italian', 'English','Japanese']
# langages_fr =['Espagnol','Chinois','Arabe','Français','Italien', 'Anglais','Japonais']

# df2 = pd.concat([df2, pd.DataFrame(columns=langages_en)])

# def langage_required(row, table_en, table_fr):

#     tables = zip(table_en, table_fr)

#     desc_low = str.lower(row['Description'])
    
#     for key_en,key_fr in tables:

#         key_en_low = str.lower(key_en)
#         key_fr_low = str.lower(key_fr)

#         if (key_en_low in desc_low)   or (key_fr_low in desc_low):
#             row[key_en] = 1
#     return row

In [164]:
# df2 = df2.apply(lambda row: langage_required(row,langages_en,langages_fr), axis=1)
# df2

In [165]:
# df2 = pd.concat([df2, pd.DataFrame(columns=['Type of contract', 'Contract pro'])])

In [166]:
# worktype_en = ['Full-time','Part-time','Remote','Internship','Contract','Temporary','Comission','Freelance','']
# worktype_fr = ['Temps plein','Temps partiel','Distance','Stage','Contrat','Temporaire','Commision','Indépendant','Apprentissage']

# def work_type(x):
#     worktypes = zip(worktype_en, worktype_fr)
#     x_lower =  str.lower(x)

#     for wtype_en,wtype_fr in worktypes:

#         wtype_en_lower = str.lower(wtype_en)
#         wtype_fr_lower = str.lower(wtype_fr)

#         if (wtype_en_lower in x_lower) or (wtype_en_lower.replace(' ','') in x_lower) or (wtype_fr_lower in x_lower) or (wtype_fr_lower.replace(' ','') in x_lower) or (wtype_en_lower.replace('-',' ') in x_lower):

#             return wtype_en

#     return np.NaN

# df2['Type of contract'] = df2['Description'].apply(lambda x : work_type(x))
# df2['Type of contract'][df2['Type of contract'].isna()] =  df2['Type de contrat'].apply(lambda x : contrat_type(x))
# df2['Type of contract'][df2['Type of contract'].isna()] =  df2['Poste'].astype(str).apply(lambda x : contrat_type(x))


In [167]:
# df2['Type of contract'].value_counts()

salaires = ['30 000 € - 40 000 € par an','40 000 € - 50 000 € par an','50 000 € - 60 000 € par an']


In [168]:
# # def key_required(x, table_en, table_fr):
# #     tables = zip(table_en, table_fr)
    
# #     for key_en, key_fr in langages:

# #         desc_low = str.lower(x)

# #         key_en_low = str.lower(key_en)
# #         key_fr_low = str.lower(key_fr)

# #         if (key_en_low in desc_low) or (key_fr_low in desc_low):

# #             row[key_en] = 1

# #     return np.Nan

# contrat_type_en = ['Open-ended','Fixed-term']
# contrat_type_fr = ['CDI','CDD']

# def contrat_type(x):
#     contrat_types = zip(contrat_type_en, contrat_type_fr)
#     x_lower =  str.lower(x)

#     for contrat_en,contrat_fr in contrat_types:

#         contrat_en_lower = str.lower(contrat_en)
#         contrat_fr_lower = str.lower(contrat_fr)

#         if (contrat_en_lower in x_lower) or (contrat_en_lower.replace(' ','') in x_lower) or (contrat_fr_lower in x_lower) or (contrat_fr_lower.replace(' ','') in x_lower) or (contrat_en_lower.replace('-',' ') in x_lower):

#             return contrat_en

#     return np.NaN

# df2['Contract pro'] = df2['Description'].apply(lambda x : contrat_type(x))
# df2['Contract pro'][df2['Contract pro'].isna()] =  df2['Type de contrat'].apply(lambda x : contrat_type(x))
# df2['Contract pro'][df2['Contract pro'].isna()] =  df2['Poste'].astype(str).apply(lambda x : contrat_type(x))

In [169]:
# df2['Contract pro'].value_counts()


In [179]:
df2

,Poste,Entreprise,Pays,Ville,Type de contrat,Salaire,Description,Python,R,SQL,...,SAAS,Java,Scikit learn,Hadoop,Hive,ML,DL,Azure,AWS,Study level required
0,Alternance - Data Scientist H/F,Malakoff Humanis,France,Paris 9e (75),"Apprentissage, Contrat pro",None,Contrat ALTERNANCE Finalité du poste Au sein d...,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bac +5
1,Data Scientist H/F,AGYLIS,France,La Défense (92),"Temps plein, CDI",40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,Master
2,DATA SCIENTIST- F/H,Ingeniance,France,La Défense (92),CDI,None,"Contexte: Vous assurez la production, la conce...",NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bac +5
3,Data Scientist and Innovation Engineer H/F,HIPAY,France,Levallois-Perret (92),None,None,Notre métier HiPay est une Fintech Européenne ...,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASSISTANT DATA SCIENTIST,Total,France,Nanterre (92),"Apprentissage, Contrat pro",None,Monter un système de reporting pour la directi...,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bac +5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,Senior Developer,VirtualMGA,USA,United States,None,None,Virtual MGA is searching for someone who can w...,NaN,1.0,1.0,...,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
737,"Senior Backend Engineer, Remote Assistance",Cruise,USA,"San Francisco, CA 94103 (South of Market area)",None,None,"We're Cruise, the self-driving ride-hailing se...",NaN,1.0,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,Master
740,Principal Software Engineer/Architect,Microsoft,USA,"Redmond, WA",None,None,Are you interested in joining a startup within...,NaN,1.0,1.0,...,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN
742,Sr. Software Engineer,"Homecare Homebase, LLC",USA,"Louisville, KY",None,None,Position Summary A Senior Software Engineer wo...,NaN,1.0,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
df2['Salaire2'] = df2.Description.str.extract(r"(\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par jour)\b|\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par mois)\b|\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par an)\b|\b(?:\d[ - ]*?){3,5}(€ par jour)\b|\b(?:\d[ - ]*?){3,5}( € par jour)\b|\b(?:\d[ - ]*?){3,5}(€ par mois)\b|\b(?:\d[ - ]*?){3,5}( € par mois)\b|\b(?:\d[ - ]*?){3,5}( € par an)\b|\b(?:\d[ - ]*?){3,5}(€ par an)\b | \$(\d){2,3}\,(\d){3}( \- )\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2}\.(\d){2}( \- )\$(\d){2}\.(\d){2}( an hour)|\$(\d){2}\.(\d){2}( an hour))")

In [162]:
df2.iloc[18,:]['Description']

'À propos LiveMentor est la référence du coaching d’entrepreneurs en France. Passionnés d\'entrepreneuriat, ils ont créé une expérience d’apprentissage d’exception pour aider toute une génération à vivre de son projet. Dans leur communauté, ils comptent 6 000 entrepreneurs, et comptent doubler en 2020, portés par une croissance inédite. Qui sont ces entrepreneurs ? Des artisans, des thérapeutes, des e-commerçants, des freelances, et bien d’autres encore. Leur ambition est simple : changer la société française en aidant chacun à vivre de sa passion. Pour cela : Le coaching est vraiment intense et surtout, les mentors savent de quoi ils parlent. Ils ne recrutent que des anciens entrepreneurs ou freelances. Chaque mentor a suivi une formation poussée, et respecte la pédagogie LiveMentor. Ils ont développé une technologie de pointe pour le coaching en ligne. Ils ont créé une belle communauté d’entrepreneurs et de porteurs de projet. Et ils en prennent soin. Elle s’exprime via des renc